In [96]:
from cv2 import add
import pandas as pd
import re

from pyparsing import rest_of_line


def add_mesage_to_df(messages, date, time, id, message, type_="message", subtype=None):
    messages.append(
        {
            "Date": date,
            "Time": time,
            "Id": id,
            "Message": message,
            "Type": type_,
            "Action_type": subtype,
        }
    )
    return messages


def manage_action(messages, current_date, current_time, id, rest_of_line):
    if "entrou usando o link de convite deste grupo" in rest_of_line:
        action_type = "entry"
        id = rest_of_line.split("entrou usando o link de convite deste grupo")[
            0
        ].strip()
    elif "saiu" in rest_of_line:
        action_type = "exit"
        id = rest_of_line.split("saiu")[0].strip()
    elif "mudou a descrição do grupo" in rest_of_line:
        action_type = "description_change"
        id = rest_of_line.split("mudou a descrição do grupo")[0].strip()
    elif "fixou uma mensagem" in rest_of_line:
        action_type = "message_pin"
        id = rest_of_line.split("fixou uma mensagem")[0].strip()
    elif "mudou as configurações" in rest_of_line:
        action_type = "group_settings_change"
        id = rest_of_line.split("mudou as configurações do grupo")[0].strip()
    elif "(arquivo anexado)" in rest_of_line:
        action_type = "file_attach"
        id = rest_of_line.split("(arquivo anexado)")[0].strip()
    elif "criou o grupo" in rest_of_line:
        action_type = "group_create"
        id = rest_of_line.split("criou o grupo")[0].strip()
    elif "foi adicionado(a)" in rest_of_line:
        action_type = "add"
        id = rest_of_line.split("foi adicionado(a)")[0].strip()
    else:
        print("Error line:", rest_of_line)
        assert False

    add_mesage_to_df(
        messages, current_date, current_time, id, None, "action", subtype=action_type
    )


def parse_whatsapp_history(file_path):
    # Lista para armazenar as mensagens extraídas
    messages = []

    # Ler o arquivo de histórico
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    # Regex para identificar a data, hora e número do remetente
    date_and_time_pattern = re.compile(r"(\d{2}/\d{2}/\d{4}) (\d{2}:\d{2}) - (.*)")

    # Variáveis para acumular mensagens
    current_date = None
    current_time = None
    current_id = None
    current_message = None

    for line in lines:
        # Verifica se a linha corresponde ao padrão de uma nova mensagem
        match = date_and_time_pattern.match(line)
        if match:
            if current_message:
                messages = add_mesage_to_df(
                    messages, current_date, current_time, current_id, current_message
                )
            current_message = ""
            current_date, current_time, rest_of_line = match.groups()
            # if "entrou usando o link de convite deste grupo" in rest_of_line:
            #     id = rest_of_line.split("entrou usando o link de convite deste grupo")[
            #         0
            #     ].strip()
            #     add_mesage_to_df(messages, current_date, current_time, id, "", "entry")
            if "‎" in rest_of_line:
                manage_action(
                    messages, current_date, current_time, current_id, rest_of_line
                )
            else:
                if ":" in rest_of_line:
                    current_id, message = rest_of_line.split(":", 1)
                    current_message = message.strip()
        else:
            current_message += "\n" + line.strip()

    messages = add_mesage_to_df(
        messages, current_date, current_time, current_id, current_message, "message"
    )

    # Criar DataFrame com as mensagens
    df = pd.DataFrame(messages)

    # Cria linha de ordem
    df["Order"] = range(1, len(df) + 1)

    return df


# Exemplo de uso
file_path = "../conversa.txt"
history = parse_whatsapp_history(file_path)
history[history["Type"] != "message"].head()

,Date,Time,Id,Message,Type,Action_type,Order
0,19/08/2024,19:53,‎~ Felipe Soares,None,action,group_create,1
4,19/08/2024,20:32,‎+55 61 9217-6339,None,action,entry,5
6,19/08/2024,20:33,‎+55 11 98910-2316,None,action,entry,7
30,19/08/2024,20:39,‎+55 11 98760-9956,None,action,entry,31
65,19/08/2024,20:44,‎+55 13 97406-4831,None,action,entry,66
